code to visualize the caustic lens

In [38]:
import os
from os.path import realpath, join

import drjit as dr
import mitsuba as mi

import numpy as np

print(mi.variants())

['scalar_rgb', 'scalar_spectral', 'cuda_ad_rgb', 'llvm_ad_rgb']


In [39]:
mi.set_variant('llvm_ad_rgb')

configs:

In [40]:
SCENE_DIR = realpath('../scenes')

CONFIGS = {
    'logo': {
        'emitter': 'gray',
        'lens': join(realpath(''), '4_outputs/logo/lens_displaced.ply'),
    },
    'qureshi': {
        'emitter': 'gray',
        'lens': join(realpath(''), '4_outputs/qureshi/lens_displaced.ply'),
    },
    'green': {
        'emitter': 'gray',
        'lens': join(realpath(''), '4_outputs/green/lens_displaced.ply'),
    },
}

config_name = 'qureshi'

config = CONFIGS[config_name]
print('selected:', config['lens'])

selected: C:\Users\Alex\Desktop\exploring-inverse-rendering\tutorials\4_outputs/qureshi/lens_displaced.ply


make hparams

In [41]:
config.update({
    'render_resolution': (256, 256),
    'spp': 32,
})

lens_fname = config['lens']

make emitter

In [42]:
emitter = {
    'type': 'directionalarea',
    'radiance': {
        'type': 'spectrum',
        'value': 0.8
    }
}

make integrator

In [43]:
integrator = {
    'type': 'ptracer',
    'samples_per_pass': 256,
    'max_depth': 4,
    'hide_emitters': False
}

make sensor

In [44]:
sensor_to_world = mi.ScalarTransform4f.look_at(
    target=[0, -20, 0],
    origin = [0, -4.65, 0],
    up = [0, 0, 1]
)
resx, resy = config['render_resolution']
sensor = {
    'type': 'perspective',
    'near_clip': 1,
    'far_clip': 1000,
    'fov': 65,
    'to_world': sensor_to_world,

    'sampler': {
        'type': 'independent',
        'sample_count': 512  # Not really used
    },
    'film': {
        'type': 'hdrfilm',
        'width': resx,
        'height': resy,
        'pixel_format': 'rgb',
        'rfilter': {
            # Important: smooth reconstruction filter with a footprint larger than 1 pixel.
            'type': 'gaussian'
        }
    }
}

make scene

In [45]:
scene = {
    'type': 'scene',
    'sensor': sensor,
    'integrator': integrator,
    # Glass BSDF
    'simple-glass': {
        'type': 'dielectric',
        'id': 'simple-glass-bsdf',
        'ext_ior': 'air',
        'int_ior': 1.5,
        'specular_reflectance': { 'type': 'spectrum', 'value': 0 },
    },
    'white-bsdf': {
        'type': 'diffuse',
        'id': 'white-bsdf',
        'reflectance': { 'type': 'rgb', 'value': (1, 1, 1) },
    },
    'black-bsdf': {
        'type': 'diffuse',
        'id': 'black-bsdf',
        'reflectance': { 'type': 'spectrum', 'value': 0 },
    },
    # Receiving plane
    'receiving-plane': {
        'type': 'obj',
        'id': 'receiving-plane',
        'filename': '../scenes/meshes/rectangle.obj',
        'to_world': \
            mi.ScalarTransform4f.look_at(
                target=[0, 1, 0],
                origin=[0, -7, 0],
                up=[0, 0, 1]
            ).scale((5, 5, 5)),
        'bsdf': {'type': 'ref', 'id': 'white-bsdf'},
    },
    # Glass slab, excluding the 'exit' face (added separately below)
    'slab': {
        'type': 'obj',
        'id': 'slab',
        'filename': '../scenes/meshes/slab.obj',
        'to_world': mi.ScalarTransform4f.rotate(axis=(1, 0, 0), angle=90),
        'bsdf': {'type': 'ref', 'id': 'simple-glass'},
    },
    # Glass rectangle, to be optimized
    'lens': {
        'type': 'ply',
        'id': 'lens',
        'filename': lens_fname,
        'to_world': mi.ScalarTransform4f.rotate(axis=(1, 0, 0), angle=0),
        'bsdf': {'type': 'ref', 'id': 'simple-glass'},
    },

    # Directional area emitter placed behind the glass slab
    'focused-emitter-shape': {
        'type': 'obj',
        'filename': '../scenes/meshes/rectangle.obj',
        'to_world': mi.ScalarTransform4f.look_at(
            target=[0, 0, 0],
            origin=[0, 15, 0],
            up=[0, 0, 1]
        ),
        'bsdf': {'type': 'ref', 'id': 'black-bsdf'},
        'focused-emitter': emitter,
    },
}

In [46]:
scene = mi.load_dict(scene)

In [47]:
params = mi.traverse(scene)
print(params['lens.vertex_positions'])
dist = -14
steps = 20

V = dr.unravel(mi.Point3f, params['lens.vertex_positions'])
W = dr.unravel(mi.Point3f, params['slab.vertex_positions'])
V.y -= dist
W.y -= dist
params['lens.vertex_positions'] = dr.ravel(V)
params['slab.vertex_positions'] = dr.ravel(W)
params.update();

for i in range(0, steps):
    V = dr.unravel(mi.Point3f, params['lens.vertex_positions'])
    W = dr.unravel(mi.Point3f, params['slab.vertex_positions'])
    V.y += dist/steps
    W.y += dist/steps
    params['lens.vertex_positions'] = dr.ravel(V)
    params['slab.vertex_positions'] = dr.ravel(W)
    params.update();
    img = mi.render(scene)
    mi.util.write_bitmap(f"5_showcase//{config_name}/{i}-{config_name}.png", img)
    mi.util.write_bitmap(f"5_showcase//{config_name}/{2*steps - i - 1}-{config_name}.png", img)

[-0.9999999403953552, -3.648938218248077e-05, -1.0, -0.9999999403953552, -0.00012660946231335402, .. 786422 skipped .., 3.417972766328603e-05, 0.9960861206054688, 0.9999999403953552, -3.657680281321518e-05, 1.0]


2024-04-06 17:27:04 WARN main [AdjointIntegrator] render(): forcing 'evaluate=true' since multi-pass rendering was requested.

2024-04-06 17:27:15 WARN main [AdjointIntegrator] render(): forcing 'evaluate=true' since multi-pass rendering was requested.

2024-04-06 17:27:26 WARN main [AdjointIntegrator] render(): forcing 'evaluate=true' since multi-pass rendering was requested.

2024-04-06 17:27:37 WARN main [AdjointIntegrator] render(): forcing 'evaluate=true' since multi-pass rendering was requested.

...

2024-04-06 17:29:52 WARN main [AdjointIntegrator] render(): forcing 'evaluate=true' since multi-pass rendering was requested.

2024-04-06 17:30:04 WARN main [AdjointIntegrator] render(): forcing 'evaluate=true' since multi-pass rendering was requested.

2024-04-06 17:30:16 WARN main [AdjointIntegrator] render(): forcing 'evaluate=true' since multi-pass rendering was requested.

2024-04-06 17:30:28 WARN main [AdjointIntegrator] render(): forcing 'evaluate=true' since multi-pass rendering was requested.

2024-04-06 17:30:39 WARN main [AdjointIntegrator] render(): forcing 'evaluate=true' since multi-pass rendering was requested.